# Justin Butler - AAE 537: HW3

This assignment was created in Jupyter Notebook and compiled using $\LaTeX$

In [1]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
%matplotlib inline

In [10]:
""" THIS CODE BLOCK CONTAINS ALL DEFINED FUNCTIONS FOR THIS ASSIGNMENT """
#Round to three decimals
def rounds(number):
    trunc = float('%.3f'% (number * 1000/1000))
    return(trunc)
#Convert from Bar to Atm
def BAC(bar):
    atm = bar * 0.986923
    return(rounds(atm))
#Isentropic total pressure
def totalPres(Ps,M, gamma=1.4):
    total = Ps * ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(rounds(total))
#Isentropic static pressure
def statPres(Pt,M,gamma=1.4):
    static = Pt / ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(rounds(static))
#Isentropic total temperature
def totalTemp(Ts,M,gamma=1.4):
    total = Ts * (1 + ((gamma-1)/2)*M**2)
    return(rounds(total))
#Isentropic static temperature
def statTemp(Tt,M,gamma=1.4):
    static = Tt / (1 + ((gamma-1)/2)*M**2)
    return(rounds(static))
#Tabulate method
def buildTable(label,array,air):
    newList = [label]
    newList.append(air)
    for item in array:
        newList.append(item)
    return(newList)

## Problem 1

To start problem 1, we will define all of the given values:

In [4]:
## Given values from problem-statement
mixRat = 3.8 #ox/fuel
P_c = 40 #atm
mDotE = 40 #kg/s
T_a = 500 #K
P_a = 1.0 #atm
M_a = 1.5

### Part i)
The first code block utilizes the CEA wrapper to generate values for the rocket flow and the air flow. The second code block determines the nozzle exit area and flow conditions required to expand the rocket exhaust to the local air pressure.

In [15]:
"""
A CEA wrapper for python requires a FORTRAN compiler,
which as of 9/25 is not installed onto the computers 
in the Aero lab. I will endeavor to install it on my 
personal computer, but for now I will include this 
section of code.
"""
##### The order of the rocket values go: #####
#####      CHAMBER - THROAT - EXIT       #####

# Static Pressures in atm
P = [BAC(40.530),BAC(23.518),BAC(1.0132)]
# Static Temperature in K
T = [3490.40, 3333.43, 1953.66]
# rho values in kg/cu m
rho = [3.1414, 1.9353, 0.14227]
# Molecular weight
mol = [22.494, 22.807, 22.807]
# C_p values in kJ/(kg * K)
C_p = [7.9264, 7.9101, 2.0456]
# ratio of specific heats
gamma = [1.1255, 1.1217, 1.2169]
# mach numbers
mach = [0.000, 1.000,  2.904]
# Total Pressures in atm
P0 = [totalPres(P[i],mach[i],gamma[i]) for i in range(len(P))]
# Total Temperature in K
T0 = [totalTemp(T[i],mach[i],gamma[i]) for i in range(len(T))]

##### Air stream values #####
T_0a = totalTemp(T_a,M_a)
P_0a = totalPres(P_a,M_a)
rho_a = 0.3626 #at T approximately 700 K

## Tabulating values (C stands for 'chart')
CT = buildTable("Temperature (K)",T,T_a)
CT0 = buildTable("Total Temp (K)",T0,T_0a)
CP = buildTable("Pressure (atm)",P,P_a)
CP0 = buildTable("Total Pres (atm)",P0,P_0a)
Cmach = buildTable("Mach",mach,M_a)
Crho = buildTable("Density (kg/cu m)",rho,rho_a)
Cmol = buildTable("Molar Mass (g / mol)",T0, 28.97 )
Cgamma = buildTable("Gamma",gamma,1.4)

table = [[" ", "Airstream", "Chamber", "Throat", "Exit" ],
         CT,CT0,CP,CP0,Cmach,Crho,Cmol,Cgamma
        ]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,Airstream,Chamber,Throat,Exit
Temperature (K),500,3490.4,3333.43,1953.66
Total Temp (K),725.0,3490.4,3536.269,3740.443
Pressure (atm),1.0,40.0,23.21,1.0
Total Pres (atm),3.671,40.0,40.006,38.243
Mach,1.5,0.0,1.0,2.904
Density (kg/cu m),0.3626,3.1414,1.9353,0.14227
Molar Mass (g / mol),28.97,3490.4,3536.269,3740.443
Gamma,1.4,1.1255,1.1217,1.2169


For the second part of ***(i)***, we want to find the area of the exit in order to properly expand the flow to the airstream pressure. 

### Part ii)

## Problem 2

For this problem, we will be solving this particular problem generally. That is, let $u_1, \rho_1, \gamma_1, T_1, \mathcal{M}_1$ be the velocity, density, ratio of specific heats, static temperature, and molar mass of the upper stream. Also let $u_2, \rho_2, \gamma_2, T_2, \mathcal{M}_2$ be the velocity, density, ratio of specific heats, static temperature, and molar mass of the lower stream. Without loss of generality, we will assume that $u_2 < u_1$ and that $\rho_2 < \rho_1$.

With this assumption and these postulates, we will set several parameters for use throughout: 
\begin{align*}
r &= \frac{u_2}{u_1}\\
s &= \frac{\rho_2}{\rho_1}\\
\lambda_s &= \frac{(1-r)(1+\sqrt{s})}{2(1+r\sqrt{s})}\\
\sigma_0 &= 11
\end{align*}
With these parameters set, we will now begin our analysis. We will begin with the differential equation of depth of the mixing layer for incompressible flow - that is:
\begin{equation}
\frac{db_i}{dx} = \frac{\sqrt{\pi}}{\sigma_0}\lambda_s
\end{equation}
This incompressible flow equation can be related to the equivalent compressible flow equation as follows:
\begin{equation}
\frac{\frac{db_c}{dx}}{\frac{db_i}{dx}} = 0.2 + 0.8e^{-3M_r^2}
\end{equation}
where 
$$M_r = \frac{2(u_1 - u_2)}{1 + \sqrt{\frac{\gamma_2\mathcal{M}_1T_2}{\gamma_1\mathcal{M}_2T_1}}}$$
From this relation between copmressible and incompressible flow, we can perform the following steps to obtain a function of the mixing depth:
\begin{align*}
\frac{\frac{db_c}{dx}}{\frac{db_i}{dx}} &= 0.2 + 0.8e^{-3M_r^2}\\
\frac{db_c}{dx} &= \frac{db_i}{dx}\left(0.2 + 0.8e^{-3M_r^2}\right)\\
b(s) &= \int_0^s \frac{db_i}{dx}\left(0.2 + 0.8e^{-3M_r^2}\right) dx\\
\end{align*}
Notice, however, that all of the terms within the integral are constant values - based on postulated values we declared at the beggining of this problem. Thus, the whole of the integral can be simplified thusly:
\begin{align*}
\int_0^s \frac{db_i}{dx}\left(0.2 + 0.8e^{-3M_r^2}\right) dx &=  \frac{db_i}{dx}\left(0.2 + 0.8e^{-3M_r^2}\right)\int_0^s dx\\
&= \left(\frac{\sqrt{\pi}}{\sigma_0}\lambda_s\right)\left(0.2 + 0.8e^{-3M_r^2}\right)\cdot s
\end{align*}
Therefore, since the question asks us to find the length where the mixing layer is 1 cm, we set $b(x) = 0.1$ meter and obtain the following general equation:
\begin{equation}
s = 0.1 \cdot \left[\left(\frac{\sqrt{\pi}}{\sigma_0}\lambda_s\right)\left(0.2 + 0.8e^{-3M_r^2}\right)\right]^{-1}
\end{equation}

<div style="text-align: right"> $\blacksquare$ </div>

## Problem 3